Graph Plot

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

path = './ARID3A_K562_ARID3A_(sc-8821)_Stanford_ZBTB7A_HepG2_ZBTB7A_(SC-34508)_HudsonAlpha-4_13/'
valid_file_list = os.listdir(path+'valid')
train_file_list = os.listdir(path+'train')

In [28]:
valid_auc_best = []
valid_loss_best = []

for i in range(len(train_file_list)):
    valid_res = []
    train_res = []

    f = open(path+'valid/'+valid_file_list[i], 'r')
    while True:
        line = f.readline()
        if not line: break
        valid_res.append(line)
    f.close()
    f = open(path+'train/'+train_file_list[i], 'r')
    while True:
        line = f.readline()
        if not line: break
        train_res.append(line)
    f.close()

    valid_auc = []
    train_auc = []
    valid_loss = []
    train_loss = []

    for epoch in range(len(valid_res)):
        res = valid_res[epoch]
        valid_auc.append(float(res.split(':')[0]))
        valid_loss.append(float(res.split(':')[1].split('\n')[0]))
        res = train_res[epoch]
        train_auc.append(float(res.split(':')[0]))
        train_loss.append(float(res.split(':')[1].split('\n')[0]))
    
    valid_auc_best.append(max(valid_auc))
    valid_loss_best.append(min(valid_loss))
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(path.split('/')[1])

    ax1.set_ylim(0.5, 1.01)
    ax1.plot(train_auc, label = 'train auc')
    ax1.plot(valid_auc, label = 'validation auc')
    ax1.legend()
    ax1.set_title('AUC')

    ax2.set_ylim(min(min(train_loss), min(valid_loss))-0.1, max(max(train_loss), max(valid_loss)+0.1))
    ax2.plot(train_loss, label = 'train loss')
    ax2.plot(valid_loss, label = 'validation loss')
    ax2.legend()
    ax2.set_title('Loss')

    if not os.path.exists(path+'graphs'):
        os.makedirs(path+'graphs')

    fig.savefig(path + 'graphs' + '/' + train_file_list[i].split(".")[0] + '.png')

    plt.close(fig)

Result

In [29]:
test_file_list = os.listdir(path+'test')

test_auc = []
experiment_id = []

for i in range(len(test_file_list)):
    f = open(path+'test/'+test_file_list[i], 'r')
    while True:
        line = f.readline()
        if not line: break
        if 'Test AUC' in line:
            test_auc.append(float(line.split(':')[1].split('\n')[0]))
            experiment_id.append(int(test_file_list[i].split('.')[0]))
    f.close()

df = pd.DataFrame({
    'valid loss' : valid_loss_best,
    'valid auc' : valid_auc_best, 
    'test auc' : test_auc
})

df.to_csv(path+'result.csv')